In [ ]:
import librosa.display
from time import sleep
import pydbus
import pyaudio
      
import serial
import time
import re
import pandas as pd
import numpy as np
import pickle
import json

from IPython.display import IFrame
import serial.tools.list_ports

import pickle
import numpy as np
from pyargus.directionEstimation import *

import math
from collections import deque
from scipy.spatial import distance

from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.ticker import MultipleLocator

import numpy as np
import matplotlib.pyplot as plt
import time
import wave
import IPython.display as ipd
import sklearn
from scipy.stats import kurtosis, skew
import os

import csv
from playsound import playsound

from kasa import Discover, SmartBulb, SmartPlug
from threading import Thread

import warnings
warnings.filterwarnings('ignore')

%run audio.ipynb
%run featurizer.ipynb
%run location.ipynb
%run utils.ipynb
%run direction.ipynb
%run WakeUp.ipynb
%run rf_attributes.ipynb

class MyThread(Thread):
    
    def __init__(self, group=None, target=None, name=None,
                 args=(), kwargs={}, Verbose=None):
        Thread.__init__(self, group, target, name, args, kwargs)
        self._return = None

    def run(self):
        if self._target is not None:
            self._return = self._target(*self._args,
                                                **self._kwargs)
    def join(self, *args):
        Thread.join(self, *args)
        return self._return

In [ ]:
path = 'data/feature.csv'

audio_runtime = 5 #sec
rf_runtime = 5
microphone_name = "Andrea" #deafult

f = Featurizer()
f.set_utils(location_id='Arena', user_id='user_01')

w = WakeUp(microphone_name)

idx = 0
while True:
    if w.wake_up():
        playsound('util/yes.wav')
        
        t1 = MyThread(target=f.get_audio_features, args=(idx, audio_runtime, microphone_name))
        t2 = MyThread(target=f.get_metadata, args=())
        t3 = MyThread(target=f.get_location, args=())
        t4 = MyThread(target=f.get_direction, args=())
        t5 = MyThread(target=f.get_rf_attributes, args=(rf_runtime,))

        t1.start()
        t2.start()
        t3.start()
        t4.start()
        t5.start()

        audio = t1.join()
        metadata = t2.join()
        coord = t3.join()
        direction = t4.join()
        rf = t5.join()
 
        row = pd.concat([metadata, coord, direction, audio, rf], axis=1)

        df = f.load_data(path)
        df = pd.concat([df, row], axis=0)

        df.to_csv(path, index=False)

        print("\n\n----------------------------")
        
        idx +=1
        
        playsound('util/okay.wav')
    
f.close_connections()

In [ ]:
# from kasa import Discover, SmartBulb, SmartPlug

# devices = await Discover.discover()
# for addr, dev in devices.items():
#     await dev.update()
#     print(f"{addr} >> {dev}")
#     print("\n\n")

In [ ]:
# from kasa import Discover, SmartBulb, SmartPlug

# plug_1 = SmartPlug("192.168.0.14")
# plug_2 = SmartPlug("192.168.0.15") 

In [ ]:
# await plug_1.turn_on()
# await plug_2.turn_on()

In [ ]:
# #if plug_1.is_on:
# await plug_1.turn_off()
# await plug_2.turn_off()

In [ ]:
f = Featurizer()
f.set_utils(location_id='Arena', user_id='user_01')


In [ ]:
while True:
    f.get_direction()
    clear_output(wait=True)